**Author:** C Mitchell

This workbook reads in the ICP-MS results and does the seawater subtraction (i.e. removing any calcium that is from salts rather than calcium carbonate).

# Initialization

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/01-ICP_MS_results_Drapeau_reformatted_20240730.csv')

In [3]:
df

,Code,Tube Number,Culture,Name,Ploidity,CaCo3,Filter,Rinse,Comment on filtration,Ca (ug/L),Mg (ug/L),Sr (ug/L),Na (ug/L)
0,MAX1,16.0,K/2,Medium only,-,No,0.8,NH4OH,NaN,49.67,3034.62,0.499,580.79
1,MAX2,17.0,K/2,Medium only,-,No,0.8,NH4OH,NaN,41.61,640.22,0.472,427.09
2,MAX3,18.0,K/2,Medium only,-,No,0.8,NH4OH,NaN,51.11,5276.07,0.495,491.48
3,MAX0,18.2,K/2,Medium only,-,No,0.8,NH4OH,NaN,50.73,5300.59,0.472,505.69
4,MBX1,19.0,K/2,Medium only,-,No,0.4,NH4OH,NaN,33.51,193.68,0.297,347.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,UAY2,84.0,RCC1472,Umbilicosphaera foliosa,2n,Yes,0.8,K2B4O7,NaN,101.97,200.98,0.786,700.22
74,UAY3,85.0,RCC1472,Umbilicosphaera foliosa,2n,Yes,0.8,K2B4O7,NaN,488.74,671.95,4.732,4161.95
75,UBY1,86.0,RCC1472,Umbilicosphaera foliosa,2n,Yes,0.4,K2B4O7,NaN,222.31,1165.15,1.384,954.29
76,UBY2,87.0,RCC1472,Umbilicosphaera foliosa,2n,Yes,0.4,K2B4O7,NaN,164.39,375.16,0.907,508.67


# PIC Calculation

## Specifiying the parameters:

The Ca2+ and Na+ concentrations in seawater (mg/L), for salinity of 35 and salinity of 38:

In [4]:
ca = 412; na = 10781
ca38 = 454; na38 = 11890

Volume parameters (ml):

In [5]:
Vfilt = 15; Vext = 5

## Calculation (aka correction steps)

Saltwater correction. The "S" cultures need to use the salinity 38 Ca2+ and Na+ numbers, whereas all the other cultures use salinity 35 numbers.

In [7]:
S_df = df[df.Code.str.startswith('S')].copy()
S_df['Ca sw corr'] = S_df['Ca (ug/L)'] - (S_df['Na (ug/L)']*ca38 / na38)

df2 = df[~df.Code.str.startswith('S')].copy()
df2['Ca sw corr'] = df2['Ca (ug/L)'] - (df2['Na (ug/L)']*ca / na)

Adding the "S" culture data back into the main df

In [8]:
df = pd.concat([df2, S_df])

Volume correction:

In [9]:
df['Ca sw + V corr'] = df['Ca sw corr']*(Vext/Vfilt)

PIC calculation:

In [10]:
df['PIC mmol/m3'] = df['Ca sw + V corr'] / 40 
df['PIC ug/l'] = df['PIC mmol/m3'] * 12

In [11]:
df

,Code,Tube Number,Culture,Name,Ploidity,CaCo3,Filter,Rinse,Comment on filtration,Ca (ug/L),Mg (ug/L),Sr (ug/L),Na (ug/L),Ca sw corr,Ca sw + V corr,PIC mmol/m3,PIC ug/l
0,MAX1,16.0,K/2,Medium only,-,No,0.8,NH4OH,NaN,49.67,3034.62,0.499,580.79,27.474890,9.158297,0.228957,2.747489
1,MAX2,17.0,K/2,Medium only,-,No,0.8,NH4OH,NaN,41.61,640.22,0.472,427.09,25.288594,8.429531,0.210738,2.528859
2,MAX3,18.0,K/2,Medium only,-,No,0.8,NH4OH,NaN,51.11,5276.07,0.495,491.48,32.327906,10.775969,0.269399,3.232791
3,MAX0,18.2,K/2,Medium only,-,No,0.8,NH4OH,NaN,50.73,5300.59,0.472,505.69,31.404865,10.468288,0.261707,3.140487
4,MBX1,19.0,K/2,Medium only,-,No,0.4,NH4OH,NaN,33.51,193.68,0.297,347.40,20.233977,6.744659,0.168616,2.023398
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23,SAY4,37.0,MIR 02,Syracosphaera pulchra,n,Yes,0.8,K2B4O7,Additional 0.8 K2B4O7.,210.86,223.56,1.938,1499.07,153.620489,51.206830,1.280171,15.362049
24,SBY1,38.0,MIR 02,Syracosphaera pulchra,n,Yes,0.4,K2B4O7,NaN,184.81,253.62,1.728,1649.04,121.844133,40.614711,1.015368,12.184413
25,SBY2,39.0,MIR 02,Syracosphaera pulchra,n,Yes,0.4,K2B4O7,NaN,220.67,220.66,1.613,1317.27,170.372222,56.790741,1.419769,17.037222
26,SAY2,35.0,MIR 02,Syracosphaera pulchra,n,Yes,0.8,K2B4O7,NaN,127.46,92.41,0.781,251.38,117.861470,39.287157,0.982179,11.786147


In [12]:
df.to_csv('data/02-ICPMS-and-PIC.csv',index=False)